<p style="font-weight:bold;"> <span style="font-size: 36px"> In Memory Initialization of unconfigured DataSource </span> </p>
<p style="font-weight:bold;"> <span style="font-size: 24px"> Reference Data </span> </p>

<a id='init-base-to-memory'></a>

In [0]:
#!import "../Constants/CalculationEngine"
#!eval calculationEngine

# Set default DataSource

In [0]:
await DataSource.SetAsync();
DataSource.Reset(x => x.ResetCurrentPartitions());

# Initialize Workspace

In [0]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());

In [0]:
Workspace.InitializeFrom(DataSource);

# Import Dimensions

In [0]:
await Import.FromFile("../Files/Dimensions.csv")
                .WithType<Novelty>()
                .WithType<AocType>()
                .WithType<PnlVariableType>()
                .WithType<BsVariableType>()
                .WithType<AmountType>()
                .WithType<DeferrableAmountType>()
                .WithType<RiskDriver>()
                .WithType<EconomicBasis>()
                .WithType<EstimateType>()
                .WithType<ValuationApproach>()
                .WithType<LineOfBusiness>()
                .WithType<OciType>()
                .WithType<LiabilityType>()
                .WithType<Profitability>()
                .WithType<Currency>()
                .WithType<Partner>()
                .WithType<CreditRiskRating>()
                .WithType<Scenario>()
                .WithType<ProjectionConfiguration>()
                .WithTarget(DataSource)
                .ExecuteAsync()

In [0]:
await Import.FromFile("../Files/Dimensions.csv").WithFormat(ImportFormats.AocConfiguration).WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromFile("../Files/ReportingNodes/ReportingNodes.csv").WithType<ReportingNode>().WithTarget(DataSource).ExecuteAsync()

# Import Parameters

In [0]:
//var log = await Import.FromFile("../Files/Parameters/YieldCurve_2020_1.csv").WithType<YieldCurve>().SnapshotMode().WithTarget(DataSource).ExecuteAsync();
//log.Merge(await Import.FromFile("../Files/Parameters/YieldCurve_2020_3.csv").WithType<YieldCurve>().SnapshotMode().WithTarget(DataSource).ExecuteAsync());
//log.Merge(await Import.FromFile("../Files/Parameters/YieldCurve_2020_12.csv").WithType<YieldCurve>().SnapshotMode().WithTarget(DataSource).ExecuteAsync());
//log.Merge(await Import.FromFile("../Files/Parameters/YieldCurve_2021_3.csv").WithType<YieldCurve>().SnapshotMode().WithTarget(DataSource).ExecuteAsync());
//log

//Iamport.DefineFormat(ImportFormats.YieldCurve, async (options, dataSet) => {
    Activity.Start();
    // Parsing Main
    var file = "../Files/Parameters/YieldCurve_2020_1.csv";
    var extension = System.IO.Path.GetExtension(file);
    var stream = await Project.FileStorage.ReadAsync(file);

    var mainTab = (await DataSetReader.ReadFromStream(stream).WithContentType(extension).ExecuteAsync()).DataSet.Tables[Main];",
    //var mainTab = dataSet.Tables[Main];
    if(mainTab == null) ApplicationMessage.Log(Error.NoMainTab);
    if(mainTab.Rows.Count() == 0) ApplicationMessage.Log(Error.IncompleteMainTab);
    if(mainTab.Columns.Where(x => x.ColumnName == nameof(YieldCurve.Year)).Count()!=1) ApplicationMessage.Log(Error.YearInMainNotFound);
    if(mainTab.Columns.Where(x => x.ColumnName == nameof(YieldCurve.Month)).Count()!=1) ApplicationMessage.Log(Error.MonthInMainNotFound);
    if(ApplicationMessage.HasErrors()) return Activity.Finish();
    
    var main = mainTab.Rows.First();
    var scenario = mainTab.Columns.Where(x => x.ColumnName == nameof(Scenario)).Count()>0? (string)main[nameof(Scenario)] : default(string);
    var year = (int)Convert.ChangeType(main[nameof(YieldCurve.Year)], typeof(int));
    var month = (int)Convert.ChangeType(main[nameof(YieldCurve.Month)], typeof(int));
    
    // Create Parsing Storage, maybe not needed
    var parsingStorage = new ParsingStorage(args, DataSource, workspace);
    await parsingStorage.InitializeAsync();
    if(Activity.HasErrors()) return Activity.Finish();
    
    var importLog = await Import.FromDataSet(dataSet)
        .WithType<YieldCurve>((dataset, datarow) => {
            var currency = datarow.Field<string>(nameof(YieldCurve.Currency));
            var values = datarow.Table.Columns.Where(c => c.ColumnName.StartsWith(nameof(YieldCurve.Values))).OrderBy(c => c.ColumnName.Length).ThenBy(c => c.ColumnName)
                                .Select(x => datarow.Field<string>(x.ColumnName).CheckStringForExponentialAndConvertToDouble()).ToArray();
            var item = new YieldCurve {
                Year = year,
                Month = month, 
                Scenario = scenario,
                Values = values
            };
            return item;
        }).WithTarget(workspace).ExecuteAsync(); 
    var oldYieldCurves = await DataSource.Query<YieldCurve>().ToArrayAsync();
    
    
    var workspace = Workspace.CreateNew();
    workspace.Initialize(x => x.FromSource(DataSource).DisableInitialization<RawVariable>().DisableInitialization<IfrsVariable>());
    
   

    if(parsingLog.Errors.Any()) return Activity.Finish().Merge(parsingLog); 
    var newYieldCurves = await workspace.Query<YieldCurve>().ToArrayAsync();
    
    newYieldCurves = newYieldCurves.Except(oldYieldCurves, YieldCurveComparer.Instance()).ToArray();
    if (!newYieldCurves.Any()){
        ApplicationMessage.Log(Warning.VariablesAlreadyImported);   
        return Activity.Finish().Merge(parsingLog);
    }
    
    await DataSource.UpdateAsync<YieldCurve>(newYieldCurves.Concat(oldYieldCurves));
    await DataSource.CommitAsync();

    var targetArgs = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().ToArrayAsync())
                                      .Where(x => x.Year == year && x.Month == month && x.Scenario == scenario)
                                      .Select(x => new ImportArgs( x.ReportingNode,
                                                                   x.Year, 
                                                                   x.Month,
                                                                   Periodicity.Quarterly,
                                                                   x.Scenario,
                                                                   ImportFormats.Cashflow)).ToArray();

    //var calculationLog = await CalculationYieldCurvesAsync(targetArgs);

    return Activity.Finish().Merge(parsingLog);//.Merge(calculationLog);
//});

In [0]:
await Import.FromFile("../Files/Parameters/ExchangeRate.csv").WithType<ExchangeRate>().SnapshotMode().WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromFile("../Files/Parameters/PartnerRating.csv").WithType<PartnerRating>().SnapshotMode().WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromFile("../Files/Parameters/CreditDefaultRate.csv").WithType<CreditDefaultRate>().SnapshotMode().WithTarget(DataSource).ExecuteAsync()

# Reset Workspace

In [0]:
Workspace.Reset(x => x.ResetInitializationRules().ResetCurrentPartitions());

In [0]:
DataSource.Query<YieldCurve>()

In [0]:
await Import.FromFile("../Files/Parameters/YieldCurve_2020_1.csv").WithType<YieldCurve>().SnapshotMode().WithTarget(DataSource).ExecuteAsync();

In [0]:
DataSource.Query<YieldCurve>()